In [3]:
from openai import OpenAI
# from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


In [4]:
import os
from dotenv import load_dotenv

In [5]:
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

create Testing Template 

In [22]:
template = """What is a good name for a company that makes {product}?"""
product = "socks"
prompt_template = PromptTemplate.from_template(template)
#prompt_template.format(product="socks")

prompt_test = PromptTemplate.from_template(template)

create new LLM, insert prompt and start chain

In [20]:
llm = OpenAI(openai_api_key=openai_api_key)

In [24]:
llm_chain = LLMChain(prompt=prompt_test, llm=llm)
llm_chain.invoke(product)

{'product': 'socks', 'text': '\n\n"Solemates Socks Co."'}

## Import Prompt

In [6]:
with open("prompt_continuouseffects.txt", "r") as file:
    prompt_continuous_effects = file.read()

In [7]:
print(prompt_continuous_effects)

"""
You will get two cards from the cardgame Magic: the Gathering. {card1} and {card2} both have continuous effects. You need to determine which effect will have which impact on the game.
You will follow these steps:

1. Give the rulestexts from both cards.
2. Check the official Magic: the Gathering rules.
2a. Look up the complete paragraph 613: Interaction of contiuous effects.
3. Determine whether the timestamp rule 613.7 occurs or the dependency rule 613.8
4. Give your judging: Which effect have which impact on the game?
5. Explain why you decided this way.
5a. Quote the rule you used for your decision.

"""


- create new LLM
- load template

In [14]:
llm = OpenAI(openai_api_key=openai_api_key)


In [20]:
prompt_template = PromptTemplate.from_template(prompt_continuous_effects)

## Case 1: Blood Moon vs. Urborg, Tomb of Yawgmoth
- Give Cardnames in prompt
- LCEL for prompt variables

In [13]:

prompt_val = prompt_template.invoke({"card1": "Blood Moon", "card2": "Tomb of Yawgmoth"})

In [10]:

llm.invoke(prompt_val)

'\n\n1. Blood Moon: Nonbasic lands are Mountains.\nTomb of Yawgmoth: Each swamp is a 1/1 black and Phyrexian creature with "Sacrifice this creature: Add {B}."\n\n2. According to the official Magic: the Gathering rules, continuous effects are those that continuously change the characteristics of objects, spells, or players. They are often created by the resolution of spells or abilities, and last until they are overwritten by a new continuous effect or the game ends. These effects can be applied in layers, with the most recently created effect being applied last.\n\n2a. In paragraph 613: Interaction of continuous effects, it is stated that if multiple continuous effects are applied to the same object, they are applied in timestamp order, meaning the most recently created effect is applied last.\n\n3. In this case, both Blood Moon and Tomb of Yawgmoth have continuous effects that affect the same game object (lands). Based on the timestamp rule in 613.7, the most recently created effect w

## Case 2: Opalescence vs Humility

In [21]:
prompt_val = prompt_template.invoke({"card1": "Opalescence", "card2": "Humility"})

In [22]:
llm.invoke(prompt_val)

"\n\n1. Opalescence - Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its converted mana cost.\n\nHumility - All creatures lose all abilities and have base power and toughness 1/1.\n\n2. According to the official Magic: the Gathering rules, continuous effects are those that continuously change the characteristics of objects or players. These effects can come from various sources, such as spells, abilities, and game rules.\n\n2a. In paragraph 613, the rules explain how multiple continuous effects interact with each other. It states that if two or more effects modify the same characteristic of an object, the one with the most recent timestamp is the one that applies.\n\n3. In this case, the timestamp rule 613.7 would apply. This is because both Opalescence and Humility have continuous effects that modify the same characteristic of creatures - their power and toughness. However, since Opalescence's effect has

In [23]:
prompt_val = prompt_template.invoke({"card1": "Humility", "card2": "Opalescence"})

In [24]:
llm.invoke(prompt_val)

'\n\n1. Humility: All creatures lose all abilities and have base power and toughness 1/1.\nOpalescence: Each other non-Aura enchantment is a creature in addition to its other types and has base power and toughness each equal to its converted mana cost.\n\n2. Rule 613: Interaction of Continuous Effects\n613.1. The continuous effects of an object are determined by the text of that object and may be modified by other continuous effects. \n613.2. Within layer 1, continuous effects are applied in a series of sublayers in the order described below. Within each sublayer, apply effects in timestamp order (see rule 613.7). Note that dependency may alter the order in which effects are applied within a sublayer. \n613.7. Within a sublayer or within an object, determine the timestamp first, then apply effects in timestamp order. Within each timestamp, apply effects in a series of sublayers in the order described above. \n613.8. Dependency may alter the order in which continuous effects are applied